In [1]:
import math
from importlib import reload
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
tf.random.set_seed(0)

In [2]:
import fBMmodel

In [24]:
reload(fBMmodel)

<module 'fBMmodel' from 'C:\\SongyanHou\\Study\\Codes\\ErgodicControl\\fBMmodel.py'>

## Finite Horizon Approach

In [25]:
# Discreize parameters
steps = 20
dt = 0.01
sqrtdt = np.sqrt(dt)
T = steps * dt
M = 1000
n = 100
H = 0.3
d = n+1

## Brownian Motion

In [26]:
X_initial = np.zeros(shape = (M,d))
model = fBMmodel.MODEL(steps,dt,T,M,n,H)
model.nn()
model.start(X_initial)
model.traindata(1)
model.build()

In [27]:
model.mainnn.compile(loss = 'mse' , optimizer = 'Adam')

In [29]:
model.train(epochs = 20, optimizer = 'Adam', verbose = 1)

Train on 1000 samples, validate on 1000 samples
Epoch 1/20
1000/1000 [==============================] - 7s 7ms/sample - loss: 23.6504 - val_loss: 5.7042
Epoch 2/20
1000/1000 [==============================] - 1s 723us/sample - loss: 6.2009 - val_loss: 4.3264
Epoch 3/20
1000/1000 [==============================] - 1s 749us/sample - loss: 5.2437 - val_loss: 3.8056
Epoch 4/20
1000/1000 [==============================] - 1s 718us/sample - loss: 5.0483 - val_loss: 3.5965
Epoch 5/20
1000/1000 [==============================] - 1s 690us/sample - loss: 4.7857 - val_loss: 3.4805
Epoch 6/20
1000/1000 [==============================] - 1s 696us/sample - loss: 4.4969 - val_loss: 3.1450
Epoch 7/20
1000/1000 [==============================] - 1s 700us/sample - loss: 4.1829 - val_loss: 3.2211
Epoch 8/20
1000/1000 [==============================] - 1s 705us/sample - loss: 4.0863 - val_loss: 2.6925
Epoch 9/20
1000/1000 [==============================] - 1s 689us/sample - loss: 3.7176 - val_loss: 2.5587

In [ ]:
model.autotrain(epo = 20, opt = MBSDEmodel.custom_optimizer)
RHO = [model.rhonn.predict(np.zeros(1))[0,0]]
for i in range(10):
    print(i+2,'th forward iteration')
    model.endtostart()
    model.autotrain(epo = 20, opt = 'Adam', verb = 1)
    print('\n\n')
    RHO.append(model.rhonn.predict(np.zeros(1))[0,0])

In [ ]:
print(RHO)
plt.semilogy(np.abs(np.array(RHO)-1),'o-')
plt.grid()
plt.show()

## Mean Reverting 

In [ ]:
# Initialize the model 

X_initial = np.zeros(shape = (M,1))
# X_initial = np.random.normal(0,np.sqrt(1/2),size = (M,1))

model = MBSDEmodel.BSDE(steps,dt,T,M)
model.nn(MBSDEmodel.initial_control_NN(),MBSDEmodel.initial_value_NN(),\
         MBSDEmodel.initial_rho_NN())
model.start(X_initial)
model.traindata(1)
model.dyna(name = 'mr')
model.build()

In [ ]:
model.autotrain(epo = 20, opt = MBSDEmodel.custom_optimizer)
RHO = [model.rhonn.predict(np.zeros(1))[0,0]]
for i in range(10):
    print(i+2,'th forward iteration')
    model.endtostart()
    model.autotrain(epo = 20, opt = 'Adam', verb = 0)
    print('\n\n')
    RHO.append(model.rhonn.predict(np.zeros(1))[0,0])

In [ ]:
rho_optimal = (2*np.sqrt(2) - 1)/2
print(RHO)
plt.semilogy(np.abs(np.array(RHO)-rho_optimal),'o-')
plt.grid()
plt.show()

In [ ]:
(2*np.sqrt(2) - 1)/2

Performence strongly depends on the distribution it starts with. Starting from the stationary distribution gives relatively good optimal cost, even though the control and value functions looks not good enough. This is because that more trainning are emphasizing on the most likely happens values. If we start with uniform distribution on interval, trainning gives good approximation in L2 sense on interval but not ideal optimal cost.


A simpler formulation using conditional expectation instead of MBSDE is also applicable. In this case, maybe a GAN structure has to be used for conditional expectation.